In [5]:
import pandas as pd
import os

# 데이터가 있는 폴더 경로
data_folder = "data"

# 각 연도별 폴더 리스트
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

# 모든 데이터를 저장할 빈 리스트
all_data = []

# 각 연도별 폴더를 순회하면서 파일을 읽어옴
for year in years:
    # 해당 연도의 폴더 경로
    year_folder = os.path.join(data_folder, year)
    
    # 해당 연도 폴더 안의 모든 파일을 읽어옴
    for file in os.listdir(year_folder):
        # 파일 경로
        file_path = os.path.join(year_folder, file)
        
        # 파일이 xls나 xlsx인 경우에만 읽어옴
        if file.endswith(".xls") or file.endswith(".xlsx"):
            # 파일을 DataFrame으로 읽어와서 리스트에 추가
            data = pd.read_excel(file_path)
            all_data.append(data)

# 모든 데이터를 하나의 DataFrame으로 결합
combined_data = pd.concat(all_data, ignore_index=True)

# 병합된 데이터 확인
print(combined_data.head())

# 병합된 데이터를 엑셀 파일로 저장
combined_data.to_excel("combined_data.xlsx", index=False)


    관측지점 Unnamed: 1       파고 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6  \
0   지점번호        지점명     유의파고       최대파고        NaN        NaN        NaN   
1    NaN        NaN    평균(m)      최대(m)       나타난날         일수        NaN   
2    NaN        NaN      NaN        NaN        NaN      ≥3.0m      ≥5.0m   
3  21229        울릉도      2.1        7.4         12         25         12   
4  22101        덕적도      1.0        4.3         18         12        NaN   

       파주기 Unnamed: 8 Unnamed: 9  ... Unnamed: 16 Unnamed: 17 Unnamed: 18  \
0  평균(sec)    최대(sec)       나타난날  ...    일평균최고(℃)        나타난날    일평균최저(℃)   
1      NaN        NaN        NaN  ...         NaN         NaN         NaN   
2      NaN        NaN        NaN  ...         NaN         NaN         NaN   
3      7.4       12.8         09  ...        10.1          05         0.1   
4      4.6        8.0         19  ...         5.1          04         0.5   

   Unnamed: 19       수온 Unnamed: 21 Unnamed: 22 Unnamed: 23 Unnamed: 24    e  
0

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
combined_data.to_excel("combined_data.xlsx", index=False)